In [1]:
import mysql.connector
creds = {
    'host' : "localhost",
    'user' : "root",
     'password' : "n3u3da!",
     'database' : "CSFoundations"
}


In [30]:
with mysql.connector.connect(**creds) as stock_db:
    cursor = stock_db.cursor(dictionary=True)
    cursor.execute("select ticker, stock_name, sum(quantity) as quantity, sum(stock_price * quantity) as value from stockdemo group by ticker, stock_name")
    res = cursor.fetchall()

In [ ]:
#Input fake entries to stock database
with mysql.connector.connect(**creds) as stock_db:
    cursor = stock_db.cursor()
    #Create dummy entries for teh portfolio
    stockinfo = [("AAPL", "Apple", "150.20", "20.76"),
                    ("GOOGL", "Alphabet", "2800.50", "12.15"),
                    ("AMZN", "Amazon", "3400.00", "12"),
                    ("MSFT", "Microsoft", "299.50", "10"),
                    ("TSLA", "Tesla", "750.00", "50"),
                    ("NFLX", "Netflix", "550.00", "45.00"),
                    ("FB", "Facebook", "3350.00", "3.50"),
                    ("NVDA", "Nvidia", "600.00", "1.80"),
                    ("DIS", "Disney", "180.00", "2.25"),
                    ("V", "Visa", "220.00", "4.00")]
    #Insert the dummy entries into the stock table
    cursor.executemany("INSERT INTO stockdemo (ticker, stock_name, stock_price, quantity) VALUES (%s, %s, %s, %s)", stockinfo)
    #Commit the changes to the database 
    stock_db.commit()